In [2]:
!pip install -U transformers
!pip install datasets
!pip install -U accelerate

In [1]:
import numpy as np
import pandas as pd
import tempfile
from datasets import load_dataset
from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer

In [4]:
pip install deepspeed==0.7.0

Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset
import pandas as pd
npc_train = load_dataset("amaydle/npc-dialogue", split="train")
npc_test = load_dataset("amaydle/npc-dialogue", split="test")

In [3]:
npc_train

Dataset({
    features: ['Name', 'Biography', 'Query', 'Response', 'Emotion'],
    num_rows: 1723
})

In [4]:
# Convert to Pandas DataFrames
npc_train_df = pd.DataFrame(npc_train)
npc_test_df = pd.DataFrame(npc_test)

In [5]:
npc_train_df['dialogue'] = npc_train_df['Query'] + ' ' + npc_train_df['Response']
grouped = npc_train_df.groupby(['Name', 'Biography'])['dialogue'].apply(lambda x: ' '.join(x)).reset_index()
grouped

,Name,Biography,dialogue
0,Adalyn the Huntress,Adalyn is a fearless and skilled hunter from t...,Have you ever hunted in a different environmen...
1,Agent 47,"Agent 47 is a genetically engineered assassin,...",Does Agent 47 have any allies? Not many that I...
2,Aiden Stone,Aiden is a skilled thief who grew up on the st...,Can you tell me about the first time you stole...
3,Alistair the Enchanter,Alistair is a powerful wizard who lived in the...,Can you tell me about a magical item you have ...
4,Althea,Althea is a half-elf ranger with a fierce inde...,"Althea, have you ever been betrayed by someone..."
...,...,...,...
97,Vincenzo,"Vincenzo is a cunning and charismatic thief, w...","What is your opinion on justice? ""Justice, for..."
98,Wednesday Addams,Wednesday Addams is a teenage goth and the dau...,"Have you ever felt scared? ""Scared, sometimes,..."
99,Zara the Brave,"Zara was born in a small village, but her curi...",How does Zara feel about working with other wa...
100,Zephyr,Zephyr is a mischievous fairy who loves playin...,Do you believe in true love? I'm not sure what...


In [6]:
data = Dataset.from_pandas(grouped)

In [14]:
# # Load the DailyDialog dataset
# # dataset = load_dataset("amaydle/npc-dialogue")

# # def concatenate_utterances(example):
# #   example['full_dialog'] = " ".join(example['Query'] + example['Response'])
# #   return example

# # dataset = dataset.map(concatenate_utterances)
# dataset = load_dataset("amaydle/npc-dialogue")
# def concatenate_utterances(example):

#   character_name = example['Name']
#   biography = example['Biography']
  
#   input_text = f"[Character: {character_name}] [Biography: {biography}] "
#   input  
#   #input_text += " ".join(example['Query'] + example['Response'])
#   input_text += example['dialogue']

#   example['full_dialog'] = input_text
# ## include the start stop tokens per character 
# ## may need to remove biography to see if it makes any differences to the output 

#   return example

# dataset = npc_train_df.apply(concatenate_utterances, axis = 1)


In [10]:
#dataset['train']['full_dialog']

In [7]:
# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('microsoft/DialoGPT-small')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('microsoft/DialoGPT-small')

In [8]:
# Encode the full dialog
def encode(examples):
  encoded = tokenizer(examples['dialogue'],
                      truncation=True,
                      padding='max_length',
                      max_length=1000,
                     return_tensors='pt')

  encoded['labels'] = encoded['input_ids'].clone()
  return encoded

# Apply functions
# dataset = data.map(concatenate_utterances, axis = 1)
encoded_dataset = data.map(encode,batched = True)

Map:   0%|          | 0/102 [00:00<?, ? examples/s]

In [19]:
print(encoded_dataset)


0      [input_ids, attention_mask, labels]
1      [input_ids, attention_mask, labels]
2      [input_ids, attention_mask, labels]
3      [input_ids, attention_mask, labels]
4      [input_ids, attention_mask, labels]
                      ...                 
97     [input_ids, attention_mask, labels]
98     [input_ids, attention_mask, labels]
99     [input_ids, attention_mask, labels]
100    [input_ids, attention_mask, labels]
101    [input_ids, attention_mask, labels]
Length: 102, dtype: object


In [16]:
import torch
print(torch.cuda.is_available())

True


In [10]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='output',
    num_train_epochs=20,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    warmup_steps=100,
    weight_decay=0.01,
    gradient_accumulation_steps = 8,
    fp16 = True,
    #deepspeed = True
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset # use 'test' for evaluation
)

In [11]:
# Evaluate before fine-tuning
pre_eval_results = trainer.evaluate(encoded_dataset)

# Get predictions for test set before fine tuning for 10 samples
pre_val_predictions = trainer.predict(encoded_dataset.select(range(10)))

In [12]:
# Fine-tune the model
trainer.train()

Step,Training Loss


TrainOutput(global_step=120, training_loss=2.808640797932943, metrics={'train_runtime': 152.4947, 'train_samples_per_second': 13.378, 'train_steps_per_second': 0.787, 'total_flos': 979845120000000.0, 'train_loss': 2.808640797932943, 'epoch': 18.82})

In [14]:
# Get predictions for validation set before fine tuning for 10 samples
pre_val_predictions = trainer.predict(encoded_dataset.select(range(10)))
# Evaluate after fine-tuning
post_eval_results = trainer.evaluate(encoded_dataset)

# Print the evaluation losses before and after fine-tuning
print('Evaluation Results before fine-tuning :', pre_eval_results['eval_loss'])
print('Evaluation Results after fine-tuning  :', post_eval_results['eval_loss'])

# Get predictions for validation set before fine tuning for 10 samples
post_val_predictions = trainer.predict(encoded_dataset.select(range(10)))

# Zip the pre and post tuning predictions
predictions = zip(pre_val_predictions.predictions, post_val_predictions.predictions)

Evaluation Results before fine-tuning : 5.628950595855713
Evaluation Results after fine-tuning  : 1.0482536554336548


In [17]:
for idx, (pre, post) in enumerate(predictions):
    pre_pred = tokenizer.decode(np.argmax(pre, axis=-1), skip_special_tokens=True)
    post_pred = tokenizer.decode(np.argmax(post, axis=-1), skip_special_tokens=True)
    ground_truth = encoded_dataset[idx]["dialogue"]
    
    print('Ground truth \n' + ground_truth + '\n')
    print('Pre-prediction \n' + pre_pred + '\n')
    print('Post-prediction \n'+ post_pred + '\n')
    print('----------------------------------------------------------------------------------------------------------------------\n')

Ground truth 
Have you ever hunted in a different environment? Yes, I have hunted in many different environments, from forests to mountains to deserts. Do you hunt for sport or food? Both, but mostly for food. What do you like most about hunting? The thrill of the chase. Have you ever encountered a magical creature during your hunts? Yes, I've hunted enchanted deer and unicorns. Have you ever been injured while hunting? Yes, I have been injured many times. Hunting can be dangerous, but I always heal quickly. Why do you hunt? Hunting provides food and resources for my village, and it keeps me sharp and skilled. Do you have any hunting tips for new hunters? My best tip is to always be prepared and stay alert. Hunting can be dangerous, so it's important to be ready for anything. What is your favorite weapon for hunting? My favorite weapon for hunting is my bow. It is precise and silent, and it allows me to take down my prey quickly and efficiently. Have you ever hunted at night? Yes, I ha